In [30]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset
from typing import Tuple
from transformer_model import TransformerModel
import math
import os
from tempfile import TemporaryDirectory
import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, 'C:/awilde/britta/LTU/AdvancedDeepLearning/Labs/Lab1/Task1')
from data_loading_code import getData



In [32]:
# get the preprocessed Data
x_train, y_train, x_val, y_val, vocabsize, word_vectorizer = getData("../Task1/amazon_cells_labelled.txt")

Train data: torch.Size([900, 7277])
Train labels: torch.Size([900])
Validation data: torch.Size([100, 7277])
Validation labels: torch.Size([100])
Vocab size: 7277
Successfully preprocessed the data


In [33]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# load data
with open('amazon_cells.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()

In [3]:
print(text[:10])

['Sentence\n', 'So there is no way for me to plug it in here in the US unless I go by a converter.\n', '"Good case, Excellent value."\n', 'Great for the jawbone.\n', 'Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!\n', 'The mic is great.\n', 'I have to jiggle the plug to get it to line up right to get decent volume.\n', '"If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one."\n', 'If you are Razr owner...you must have this!\n', '"Needless to say, I wasted my money."\n']


In [16]:
print(len(text))
print(int(0.8*len(text)))
train_data = text[:int(0.8*len(text))]
val_data = text[int(0.8*len(text)):]

1001
800


In [17]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens():
    for sentence in train_data:
        tokens = tokenizer(sentence)
        yield tokens

In [18]:
token_generator = yield_tokens()
vocab = build_vocab_from_iterator(token_generator)
vocab.get_stoi()

{'costs': 924,
 '.': 0,
 'but': 32,
 'forget': 551,
 'unhappy': 676,
 '18': 711,
 'that': 23,
 'seems': 631,
 'carriers': 850,
 'rip': 1457,
 'exchanged': 1037,
 '!': 12,
 'a': 7,
 'to': 9,
 '10': 704,
 'or': 61,
 'continue': 913,
 'message': 1255,
 'all': 35,
 'pc': 426,
 'way': 213,
 'every': 224,
 'echo': 1008,
 ')': 59,
 'i': 2,
 'awesome': 242,
 'the': 1,
 'face': 389,
 'much': 175,
 'conversation': 511,
 'and': 3,
 'docking': 524,
 'potentially': 1371,
 'leather': 412,
 'comfortably': 506,
 ',': 4,
 'charm': 292,
 'on': 21,
 'thanks': 1584,
 'decent': 378,
 'motorola': 137,
 'toast': 1602,
 'couldn': 186,
 'pics': 1350,
 'didn': 293,
 'it': 5,
 'shield': 1488,
 'used': 114,
 'able': 358,
 'is': 6,
 'bother': 495,
 'plenty': 1361,
 'amazed': 484,
 'headset': 33,
 'peeling': 1339,
 'night': 1293,
 'very': 19,
 'monkeys': 1268,
 'keeps': 1191,
 'distracting': 972,
 'hs850': 1138,
 'this': 8,
 'item': 89,
 'if': 45,
 'backlight': 793,
 'division': 974,
 'buttons': 247,
 "'": 10,
 'na

In [19]:
vocab['the']

1

In [20]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [21]:
train_data = data_process(train_data)
val_data = data_process(val_data)
print(train_data.shape)
print(val_data.shape)

RuntimeError: Token talking not found and default index is not set

In [12]:
print(train_data.shape)

torch.Size([612, 20])


In [13]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape ``[full_seq_len, batch_size]``
        i: int

    Returns:
        tuple (data, target), where data has shape ``[seq_len, batch_size]`` and
        target has shape ``[seq_len * batch_size]``
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [14]:
ntokens = len(vocab)  # size of vocabulary
print(ntokens)
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 2  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 2  # number of heads in ``nn.MultiheadAttention``
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

1904


c:\Users\britta\anaconda3\envs\nnlm\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module, epoch) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        print(data)
        print(targets)
        output = model(data)
        output_flat = output.view(-1, ntokens)
        loss = criterion(output_flat, targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            output = model(data)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
best_val_loss = float('inf')
epochs = 3

with TemporaryDirectory() as tempdir:
    best_model_params_path = os.path.join(tempdir, "best_model_params.pt")

    for epoch in range(1, epochs + 1):
        epoch_start_time = time.time()
        train(model, epoch)
        val_loss = evaluate(model, val_data)
        val_ppl = math.exp(val_loss)
        elapsed = time.time() - epoch_start_time
        print('-' * 89)
        print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
            f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
        print('-' * 89)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_params_path)

        scheduler.step()
    model.load_state_dict(torch.load(best_model_params_path)) # load best model states

In [ ]:
# test_loss = evaluate(model, test_data)
# test_ppl = math.exp(test_loss)
# print('=' * 89)
# print(f'| End of training | test loss {test_loss:5.2f} | '
#       f'test ppl {test_ppl:8.2f}')
# print('=' * 89)